# NER Spacy auf Deutsch
Wenden sie das Named Entity Recognition auf deutsche Texte an, zum Beispiel auf Ausschnitte aus den Zeitungsartikeln oder den Schweizer Schlagzeilen.

Wie beurteilen sie die Ergebnisse?

In [3]:
#python -m spacy download de_core_news_sm

import spacy
from spacy import displacy
import re
import nltk

# python -m spacy download de_core_news_sm

text = """
Finnland hat bis auf einen alle Grenzübergänge zu Russland vollständig geschlossen, da trotz Minusgraden immer mehr Menschen ankommen. Helsinki wirft Moskau vor, diese Migration gezielt zu steuern.
"""

text = re.sub(r'\n', '', text)

nlp = spacy.load('de_core_news_sm')
text_nlp = nlp(text)

displacy.render(text_nlp, style='ent', jupyter=True)

C:\Users\yhutter\GitRepos\fhgr-ta\code\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\yhutter\GitRepos\fhgr-ta\code\venv\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\yhutter\GitRepos\fhgr-ta\code\venv\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# Aufgabe: NER auf Deutsch selbst implementiert - Daten einlesen

Der Ansatz, NER selbst zu implementieren und mittels eines annotierten Datensatzes zu trainieren lässt sich auch ins Deutsche übertragen.

Dazu habe ich einen annotierten Datensatz gefunden, siehe Link im Moodle. (GermEval2014)

Leider ist dieser Datensatz leicht unterschiedlich aufgebaut, so dass das in der Vorlesung vorgeführte Verfahren (wie üblich) wieder angepasst werden muss. Dazu einige Hinweise:

* Laden sie sich den Datensatz herunter
* Öffnen sie den kleinsten Datensatz ("dev") in einem Editor.
* Was fällt ihnen an der Struktur auf?
* Was fehlt und was muss beim Einlesen beachtet werden?
* Lesen sie die Daten ein und versuchen sie, die Daten analog zum Vorbild in einem DataFrame zu speichern.  (Wie schon angedeutet, muss die Struktur angepasst werden.)

## Daten einlesen

In [4]:
import csv

sentences = []
sentence_number = -1
with open("./NER-de-dev.tsv", "r", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        # Skip empty rows
        if len(row) == 0:
            continue
            
        if row[0] == '#':
            # Beginning of new line
            sentence_number += 1
        else:
            word = row[1]
            ner_tag = row[2]
            word_information = (sentence_number, word, ner_tag)
            sentences.append(word_information)
    sentences.append(word_information)

sentences

[(0, 'Gleich', 'O'),
 (0, 'darauf', 'O'),
 (0, 'entwirft', 'O'),
 (0, 'er', 'O'),
 (0, 'seine', 'O'),
 (0, 'Selbstdarstellung', 'O'),
 (0, '\tO\tO\n8\tEcce\tB-OTH\tO\n9\thomo\tI-OTH\tO\n10\t', 'O'),
 (0, 'in', 'O'),
 (0, 'enger', 'O'),
 (0, 'Auseinandersetzung', 'O'),
 (0, 'mit', 'O'),
 (0, 'diesem', 'O'),
 (0, 'Bild', 'O'),
 (0, 'Jesu', 'B-PER'),
 (0, '.', 'O'),
 (1, '1980', 'O'),
 (1, 'kam', 'O'),
 (1, 'der', 'O'),
 (1, 'Crown', 'B-OTH'),
 (1, 'als', 'O'),
 (1, 'Versuch', 'O'),
 (1, 'von', 'O'),
 (1, 'Toyota', 'B-ORG'),
 (1, ',', 'O'),
 (1, 'sich', 'O'),
 (1, 'in', 'O'),
 (1, 'der', 'O'),
 (1, 'Oberen', 'O'),
 (1, 'Mittelklasse', 'O'),
 (1, 'zu', 'O'),
 (1, 'etablieren', 'O'),
 (1, ',', 'O'),
 (1, 'auch', 'O'),
 (1, 'nach', 'O'),
 (1, 'Deutschland', 'B-LOC'),
 (1, '.', 'O'),
 (2, '–', 'O'),
 (2, '4:26', 'O'),
 (2, '#', 'O'),
 (2, 'Sometime', 'B-OTH'),
 (2, 'Ago/La', 'I-OTH'),
 (2, 'Fiesta', 'I-OTH'),
 (2, '–', 'O'),
 (2, '23:18', 'O'),
 (2, 'Alle', 'O'),
 (2, 'Stücke', 'O'),
 (2, 'wu

# Sätze auffüllen

In [5]:
import pandas as pd
df = pd.DataFrame.from_records(sentences, columns=['Sentence', 'Word', "Tag"])
df.head()

C:\Users\yhutter\AppData\Local\Temp\ipykernel_12432\3066147842.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Sentence,Word,Tag
0,0,Gleich,O
1,0,darauf,O
2,0,entwirft,O
3,0,er,O
4,0,seine,O


In [6]:
df = df.fillna(method='ffill')  # forward fill
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24263 entries, 0 to 24262
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  24263 non-null  int64 
 1   Word      24263 non-null  object
 2   Tag       24263 non-null  object
dtypes: int64(1), object(2)
memory usage: 568.8+ KB


C:\Users\yhutter\AppData\Local\Temp\ipykernel_12432\1766959874.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')  # forward fill


# Daten in korrekte Struktur bringen

In [7]:
agg_func = lambda s: [(w, t) for w, t in zip(s['Word'].values.tolist(), s['Tag'].values.tolist())]

grouped_df = df.groupby('Sentence').apply(agg_func)  # Gruppieren anhand der Satznummer (also jeder Satz für sich)

# Print first sentence
print(grouped_df[grouped_df.index == 0].values)

[list([('Gleich', 'O'), ('darauf', 'O'), ('entwirft', 'O'), ('er', 'O'), ('seine', 'O'), ('Selbstdarstellung', 'O'), ('\tO\tO\n8\tEcce\tB-OTH\tO\n9\thomo\tI-OTH\tO\n10\t', 'O'), ('in', 'O'), ('enger', 'O'), ('Auseinandersetzung', 'O'), ('mit', 'O'), ('diesem', 'O'), ('Bild', 'O'), ('Jesu', 'B-PER'), ('.', 'O')])]


C:\Users\yhutter\AppData\Local\Temp\ipykernel_12432\505240097.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = df.groupby('Sentence').apply(agg_func)  # Gruppieren anhand der Satznummer (also jeder Satz für sich)


# Features und Training
Nach dem Einlesen der Daten passen sie auch das Feature Engineering an. Nicht vorhandene Daten können nicht als Features verwendet werden. Sobald das Feature Engineering funktioniert, trainieren sie den CRF-Algorithmus.

Übernehmen sie auch die Auswertung und geben sie einige Beispielsätze aus.

In [8]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        # das Wort an der aktuellen Position
        # 'bias': 1.0, HR: scheint keine Rolle zu spielen
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        # das Wort davor (falls vorhanden)
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        # das Wort danach (falls vorhanden)
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2nertag(sent):
    return [ner_tag for token, ner_tag in sent]

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [9]:
sentences = [s for s in grouped_df]
sent2features(sentences[0][5:7])

[{'word.lower()': 'selbstdarstellung',
  'word[-3:]': 'ung',
  'word[-2:]': 'ng',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'O',
  'postag[:2]': 'O',
  'BOS': True,
  '+1:word.lower()': '\to\to\n8\tecce\tb-oth\to\n9\thomo\ti-oth\to\n10\t',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'O',
  '+1:postag[:2]': 'O'},
 {'word.lower()': '\to\to\n8\tecce\tb-oth\to\n9\thomo\ti-oth\to\n10\t',
  'word[-3:]': '10\t',
  'word[-2:]': '0\t',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'O',
  'postag[:2]': 'O',
  '-1:word.lower()': 'selbstdarstellung',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:postag': 'O',
  '-1:postag[:2]': 'O',
  'EOS': True}]

In [10]:
print(sent2nertag(sentences[0]))

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O']


# Modell trainieren

In [11]:
from sklearn.model_selection import train_test_split
import numpy as np

X = np.array([sent2features(s) for s in sentences], dtype=object)
Y = np.array([sent2nertag(s) for s in sentences], dtype=object)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape

((972,), (325,))

In [12]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
                           c1=0.1,
                           c2=0.1,
                           max_iterations=100,
                           all_possible_transitions=True,
                           verbose=True)

In [116]:
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 972/972 [00:00<00:00, 11672.39it/s]


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 25299
Seconds required: 0.021

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.03  loss=18512.88 active=25262 feature_norm=1.00
Iter 2   time=0.01  loss=8924.01  active=25052 feature_norm=2.73
Iter 3   time=0.01  loss=8249.21  active=24746 feature_norm=2.60


Iter 4   time=0.03  loss=6050.10  active=13857 feature_norm=2.38
Iter 5   time=0.01  loss=5176.20  active=14324 feature_norm=2.80
Iter 6   time=0.01  loss=2945.52  active=12522 feature_norm=5.32
Iter 7   time=0.01  loss=1967.58  active=10054 feature_norm=7.05
Iter 8   time=0.01  loss=1245.62  active=8691  feature_norm=8.82
Iter 9   time=0.01  loss=772.31   active=8129  feature_norm=11.16
Iter 10  time=0.01  loss=463.12   active=7196  feature_norm=14.28
Iter 11  time=0.01  loss=314.79   active=6830  feature_norm=16.42
Iter 12  time=0.01  loss=240.07   active=6521  feature_norm=18.08
Iter 13  time=0.01  loss=201.30   active=6064  feature_norm=19.01
Iter 14  time=0.01  loss=160.28   active=5561  feature_norm=20.81
Iter 15  time=0.01  loss=130.42   active=3983  feature_norm=21.80
Iter 16  time=0.01  loss=113.17   active=2946  feature_norm=22.55
Iter 17  time=0.01  loss=103.85   active=1983  feature_norm=22.86
Iter 18  time=0.01  loss=98.52    active=1519  feature_norm=22.86
Iter 19  time=0

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    max_iterations=100, verbose=True)

In [117]:
from sklearn_crfsuite import metrics as crf_metrics
y_pred = crf.predict(X_test)
crf_metrics.flat_accuracy_score(y_test, y_pred)

0.9998368944707225

# Modell testen

In [118]:
y_pred = crf.predict(X_test)
print(y_pred[1])

['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'B-ORG', 'O']


In [119]:
# zum Vergleich
print(y_test[1])

['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'B-ORG', 'O']


# Konfussionsmatrix

In [120]:
from collections import defaultdict

def confusion_matrix(y_true, y_pred):
    """
    Konfusionsmatrix für den Datentyp hier: Liste von Listen mit variabler Länge
    Label werden on-the-fly angelegt, anhand von y_true
    Rückgabe: Dictionary echtes_label: (Dictionary vorhergesagtes_label: Anzahl)
    :param y_true: 
    :param y_pred: 
    :return: 
    """
    cm = defaultdict(lambda: defaultdict(int))
    for seq1, seq2 in zip(y_true, y_pred):
        for l1, l2 in zip(seq1, seq2):
            cm[l1][l2] += 1
    return cm

In [121]:
conf_mat = confusion_matrix(y_test, y_pred)
cnf_df = pd.DataFrame(conf_mat)
cnf_df

,O,B-PER,I-PER,B-LOC,B-ORG,I-ORG,B-ORGpart,B-LOCpart,B-OTH,I-OTH,I-LOC,B-LOCderiv,B-OTHpart,B-PERpart,B-ORGderiv,B-OTHderiv,I-LOCderiv
O,5573.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B-PER,NaN,113.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I-PER,NaN,NaN,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B-LOC,NaN,NaN,NaN,102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
B-ORG,NaN,NaN,NaN,NaN,59.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I-ORG,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B-ORGpart,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B-LOCpart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B-OTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I-OTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
